In [44]:
%load_ext autoreload
%autoreload 2

import os
from datetime import datetime
import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.utils.tensorboard import SummaryWriter

from market_dynamics import bs_delta, bs_generator, bs_call_price
from data import SimulationData
from utils import call_payoff, stochastic_integral
from models import ControlNet
from train import train, test

import seaborn as sns

current_time = datetime.now().strftime('%b%d_%H-%M-%S')
comment = ""
log_dir = os.path.join('experiments', current_time + '_' + comment)
writer = SummaryWriter(log_dir)

n_simulations = 500 
initial_value = 100
sigma = 0.2
n_steps = 50
rf = 0
strike = 95
fc_dims = [16, 16]
LR = 0.01
EPOCHS = 20
p_train = 0.9

bs_params = {"n_simulations": n_simulations,
             "n_steps": n_steps,
             "initial_value": initial_value,
             "sigma": sigma,
             }

price_params = {"n_steps": n_steps,
                "initial_value": initial_value,
                "sigma": sigma,
                "rf": rf,
                "strike": strike,
                }

payoff_params = {"strike": strike}

n_train = int(p_train * n_simulations)
n_test = n_simulations - n_train

dt = SimulationData(bs_generator, bs_params, bs_call_price, price_params, call_payoff, payoff_params)
dt_train, dt_test = torch.utils.data.random_split(dt, [n_train, n_test])
data_loader_train = DataLoader(dt_train, batch_size=20, shuffle=True)
cn = ControlNet(n_steps, 1, fc_dims, 1)

optimizer = Adam(cn.parameters(), lr=LR)
criterion = torch.nn.MSELoss()
train(data_loader_train, cn, criterion , optimizer, EPOCHS, writer)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Epoch 0: : 23batch [00:00, 41.83batch/s, loss=5.78]
Epoch 1: : 23batch [00:00, 42.49batch/s, loss=3.87]
Epoch 2: : 23batch [00:00, 42.28batch/s, loss=1.66]
Epoch 3: : 23batch [00:00, 43.23batch/s, loss=1.91]
Epoch 4: : 23batch [00:00, 43.64batch/s, loss=3.32]
Epoch 5: : 23batch [00:00, 42.85batch/s, loss=0.554]
Epoch 6: : 23batch [00:00, 43.89batch/s, loss=2.98]
Epoch 7: : 23batch [00:00, 43.92batch/s, loss=2.46]
Epoch 8: : 23batch [00:00, 42.62batch/s, loss=2.5] 
Epoch 9: : 23batch [00:00, 43.57batch/s, loss=2.99]
Epoch 10: : 23batch [00:00, 41.74batch/s, loss=1.81]
Epoch 11: : 23batch [00:00, 42.93batch/s, loss=2.26]
Epoch 12: : 23batch [00:00, 42.61batch/s, loss=2]   
Epoch 13: : 23batch [00:00, 42.65batch/s, loss=1.85]
Epoch 14: : 23batch [00:00, 42.43batch/s, loss=2.57]
Epoch 15: : 23batch [00:00, 43.13batch/s, loss=2.13] 
Epoch 16: : 23batch [00:00, 43.32batch/s, loss=3.41]
Epoch 17: : 23batch [00:00, 40.61batch/s, loss=2.76]
Epoch 18: : 23batch [00:00, 38.54batch/s, loss=1.91]
E

In [45]:
def delta(x):
    return bs_delta(n_steps, x, sigma, rf, strike)

In [46]:
# Evaluation In-Sample

_ ,model_loss = test(data_loader_train, cn, criterion)

l = len(data_loader_train.dataset)
x, x_inc, payoff, price = data_loader_train.dataset[:l]

d_hedge = x.apply_(delta)
si = stochastic_integral(x_inc, d_hedge)
delta_loss = criterion(price + si, payoff)

print(model_loss)
print(delta_loss)

tensor(1.5993, grad_fn=<MseLossBackward>)
tensor(2.8343)


In [47]:
# Evaluation Out-Sample
data_loader_test = DataLoader(dt_test, batch_size=20, shuffle=True)

_ ,model_loss = test(data_loader_test, cn, criterion)

l = len(data_loader_test.dataset)
x, x_inc, payoff, price = data_loader_test.dataset[:l]

d_hedge = x.apply_(delta)
si = stochastic_integral(x_inc, d_hedge)
delta_loss = criterion(price + si, payoff)

print(model_loss)
print(delta_loss)

tensor(1.3253, grad_fn=<MseLossBackward>)
tensor(3.0624)
